In [1]:
%pwd  #curent woking directory

'c:\\bokhtiar_ahmed_bappy\\medi_bot\\research'

In [2]:
import os
os.chdir("../") #one folder back

In [3]:
%pwd

'c:\\bokhtiar_ahmed_bappy\\medi_bot'

In [4]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader  #pypdfloader: to load the document and directoryloader: to load from directory
from langchain.text_splitter import RecursiveCharacterTextSplitter # for chucking operation

In [5]:
# Extract the data from PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader
                             )
    documents = loader.load()
    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
#extracted_data

In [7]:
# split the data to make chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 7020


In [10]:
#text_chunks

In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [10]:
# download the embeddings from hugging face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\mohit bhade\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("Hello World")
print('Length',len(query_result))

Length 384


In [13]:
#query_result

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    name = index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)



In [ ]:
#! pip install langchain-google-genai
#! pip install --upgrade langchain-google-genai



In [17]:
import os

#from langchain_google_genai import genai
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [18]:
# embed the chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch =PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding=embeddings,
)

In [21]:
# load Existing index 

In [19]:
docsearch

In [20]:
retriever =docsearch.as_retriever(search_type = "similarity",search_kwargs = {"k":3})

In [21]:
retriever_docs = retriever.invoke("what is acne?")

In [22]:
retriever_docs

[Document(id='3fc59977-99e5-4a79-b5ab-15ffe9d16232', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='ce38a6a7-0ab6-4261-b8f4-6997746ae79e', metadata={'page': 40.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Cliffs, NJ: Prentice Hall, 1995.\nGoldstein, Sanford M., and Richard B. Odom. “Skin &\nAppendages: Pustular Disorders.” In Current Medical\nDiagnosis and Treatment, 1996. 35th ed. Ed. Stephen\nMcPhee, et al. Stamford: Appleton & Lange, 1995.\nKaptchuk, Ted J., Z’ev Rosenberg, and K’an Herb Co., Inc.\nK’an Herbals: Formulas by Ted Kaptchuk, O.M.D. San\nFrancisco: Andrew Miller, 1996.\nPERIODICALS\n“Adult Acne.” Harvard Wom

In [ ]:
#! pip install langchain-google-genai


In [23]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


In [24]:
def load_hugging_face_model(model_name="gpt2"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return generator

# Initialize Hugging Face generator model
generator = load_hugging_face_model()

# Generate answer using the model
def generate_answer(generator, input_text):
    response = generator(input_text, max_length=1000, num_return_sequences=1)
    return response[0]['generated_text']

# Define the question-answering process using the Hugging Face model
def custom_question_answer_chain(input_text):
    return generate_answer(generator, input_text)

# Test the question-answer chain
response_text = custom_question_answer_chain("What is Acromegaly and gigantism?")
print(response_text)


C:\Users\mohit bhade\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mohit bhade\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `trunca

What is Acromegaly and gigantism?

Acromegaly is the inability to see light. To do so, a person is physically incapable of seeing light. This creates a disorder called "darkness blindness". Often called the "darkness deficit", which may occur as an individual looks upon light that isn't as bright as the light reflects off their own body, such as fluorescent light or fluorescent light emitted by pets. If your light is not as bright as the outside world


In [25]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are assistant for question-answering tasks."
    "Use the following pieces of retrieved context ot ansswer"
    "the question.If you don't know the answer,say that you"
    "answer consise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [26]:
# Using the Hugging Face generator instead of OpenAI
def rag_chain(input_text):
    retrieved_documents = retriever.get_relevant_documents(input_text)
    context = " ".join([doc.page_content for doc in retrieved_documents])
    prompt_with_context = system_prompt.format(context=context)
    
    response = custom_question_answer_chain(prompt_with_context)
    return response

In [27]:
response = rag_chain("What is Acromegaly and gigantism?")
print(response)

C:\Users\mohit bhade\AppData\Local\Temp\ipykernel_5496\310831556.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = retriever.get_relevant_documents(input_text)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


ValueError: Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.